In [1]:

from models import *
from config import *

from datasets import *

import math

dataset = ToyData("train")

idx = [4, 7, 9, 11]
ims = torch.cat([dataset[i]["image"].unsqueeze(0) for i in idx])

print(ims.shape)

percept = PSGNet(config.imsize,3)


outputs = percept(ims)

def normalize_outputs(outputs):
    recons = outputs["recons"]
    B = recons[0].shape[0]
    W = int(math.sqrt(recons[0].shape[1]))

    clusters  = outputs["clusters"]
    features  = outputs["features"]
    centroids = outputs["centroids"]
    moments   = outputs["moments"]

    for item in clusters:print(item[0].shape, item[1].shape)

    level_reconstructions = [item.reshape([B,W,W,3]) for item in recons]

    return {"recons":level_reconstructions, "clusters": clusters, "features": features, "centroids": centroids, "moments": moments}

outputs = normalize_outputs(outputs)
recons = outputs["recons"]

for i,item in enumerate(recons):print("level:{}".format(i),item.shape)



class AbstractNet(nn.Module):
    def __init__(self,config):
        super().__init__()
        
        self.num_heads =8
        self.feature_decoder = nn.Transformer(nhead=16, num_encoder_layers=12,d_model = config.global_feature_dim,batch_first = True)
        self.spatial_decoder = nn.Transformer(nhead=16, num_encoder_layers=12,d_model = config.global_feature_dim,batch_first = True)
        self.source_heads = nn.Parameter(torch.randn([self.num_heads,config.global_feature_dim]))

        self.coordinate_decoder = nn.Linear(config.global_feature_dim, 2)
    
    def forward(self, feature, spatial):
        B, M, C = feature.shape
        N = self.num_heads
        # [Feature Propagation]
        component_features = feature
        component_spaitals = spatial

        # [Decode Proposals]
        global_feature = torch.randn()
        source_heads = self.souce_heads
        feature_proposals = self.feature_decoder(source_heads,global_feature)
        spaital_proposals = self.spatial_decoder(source_heads,global_feature)

        # [Component Matching]
        # component_features : [B,M,C]
        # feature_proposals  : [B,N,C]

        match = torch.softmax(torch.einsum("bnc,bmc -> bnm",component_features, proposal_features)/math.sqrt(C), dim = -1)
        existence = torch.max(match, dim = 1).values  # [B, N, 1]

        # [Construct Representation]
        output_graph = 0

        return output_graph

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/melkor/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


torch.Size([4, 128, 128, 3])
torch.Size([65536]) torch.Size([65536])
torch.Size([2140]) torch.Size([2140])
level:0 torch.Size([4, 128, 128, 3])
level:1 torch.Size([4, 128, 128, 3])


In [4]:
print("clusters")
clusters = outputs["clusters"]
for item in clusters:
    nodes = item[0]
    batch = item[1]
    print(nodes.shape, batch.shape)


print("features")
features = outputs["features"]
for item in features:
    print(item.shape)


print("centroids")
features = outputs["centroids"]
for item in features:
    print(item.shape)

print("moments")
features = outputs["moments"]
for item in features:
    print(item.shape)



clusters
torch.Size([65536]) torch.Size([65536])
torch.Size([2140]) torch.Size([2140])
features
torch.Size([2140, 64])
torch.Size([869, 64])
centroids
torch.Size([2140, 2])
torch.Size([869, 2])
moments
torch.Size([2140, 2])
torch.Size([869, 2])


In [22]:
def to_dense_features(outputs):
    clusters = outputs["clusters"]
    features = outputs["features"]
    centroids = outputs["centroids"]
    moments = outputs["moments"]
    for i in range(len(clusters)):
        print("level:",i)
        sparse_feature = features[i]
        sparse_centroid = centroids[i]
        sparse_moment = moments[i]
        nodes = clusters[i][0];batch = clusters[i][1]

        #cast_batch = to_dense_batch(nodes,batch)[0]
        cast_batch = clusters[i+1][1]

        print(sparse_feature.shape,sparse_centroid.shape, sparse_moment.shape)

        feature,  batch = to_dense_batch(features[i],cast_batch)
        centroid, batch = to_dense_batch(centroids[i],cast_batch)
        moment,   batch = to_dense_batch(moments[i],cast_batch)

        print(feature.shape, centroid.shape, moment.shape)


to_dense_features(outputs)

level: 0
torch.Size([2140, 64]) torch.Size([2140, 2]) torch.Size([2140, 2])
torch.Size([4, 547, 64]) torch.Size([4, 547, 2]) torch.Size([4, 547, 2])
level: 1


IndexError: list index out of range